In [1]:
import ee
import geemap
import pandas as pd
import os
from datetime import datetime
from tqdm import tqdm 
import time
from datetime import datetime

In [2]:
geemap.set_proxy(port=7890) # set your proxy port
ee.Authenticate()
ee.Initialize(project='socd-liuziyan') # change your GEE project name

In [3]:
fishnet = ee.FeatureCollection("projects/socd-liuziyan/assets/fishnet_grid_mountain_50km_with_id")

selected_grid = fishnet.filter(ee.Filter.eq('GRID_ID', 10882))

# 创建地图并显示结果
Map = geemap.Map()
Map.addLayer(fishnet, {'color': 'blue'}, 'All Fishnet Grids')
Map.addLayer(selected_grid, {'color': 'red'}, 'Selected Grid')
Map.centerObject(selected_grid, 10)
Map

Map(center=[38.749798086673444, 100.25000035704018], controls=(WidgetControl(options=['position', 'transparent…

In [6]:
# 加载 ALOS DEM 数据
dataset = ee.ImageCollection('JAXA/ALOS/AW3D30/V4_1')
elevation_collection = dataset.select('DSM')
proj = elevation_collection.first().select(0).projection()
elevation = elevation_collection.mosaic().setDefaultProjection(proj)

# 计算坡度和坡向
slope = ee.Terrain.slope(elevation)
aspect = ee.Terrain.aspect(elevation)

# 裁剪到选中的网格区域
elevation_clipped = elevation.clip(selected_grid)
slope_clipped = slope.clip(selected_grid)
aspect_clipped = aspect.clip(selected_grid)

# 设置下载参数
download_dir = r'D:\0_phd_papers\2_SOC_Aspect\9_plot\plot_r_distribution\plot_method\dem_ings'
os.makedirs(download_dir, exist_ok=True)

# 下载三张单独的影像
# 1. 下载高程数据
geemap.download_ee_image(
    image=elevation_clipped,
    filename=os.path.join(download_dir, 'elevation_grid_10882.tif'),
    scale=30,
    region=selected_grid.geometry()
)

# 2. 下载坡度数据
geemap.download_ee_image(
    image=slope_clipped,
    filename=os.path.join(download_dir, 'slope_grid_10882.tif'),
    scale=30,
    region=selected_grid.geometry()
)

# 3. 下载坡向数据
geemap.download_ee_image(
    image=aspect_clipped,
    filename=os.path.join(download_dir, 'aspect_grid_10882.tif'),
    scale=30,
    region=selected_grid.geometry()
)

print(f"数据已下载到: {download_dir}")
print("已生成三个单独的文件:")
print("- elevation_grid_10882.tif (高程)")
print("- slope_grid_10882.tif (坡度)")
print("- aspect_grid_10882.tif (坡向)")

elevation_grid_10882.tif: |          | 0.00/6.89M (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None


slope_grid_10882.tif: |          | 0.00/13.8M (raw) [  0.0%] in 00:00 (eta:     ?)

aspect_grid_10882.tif: |          | 0.00/13.8M (raw) [  0.0%] in 00:00 (eta:     ?)

数据已下载到: D:\0_phd_papers\2_SOC_Aspect\9_plot\plot_r_distribution\plot_method\dem_ings
已生成三个单独的文件:
- elevation_grid_10882.tif (高程)
- slope_grid_10882.tif (坡度)
- aspect_grid_10882.tif (坡向)


In [9]:
soc_data = ee.FeatureCollection('projects/socd-liuziyan/assets/1_final_socd_0_100cm')

embedding_dataset = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')
embedding_2024 = embedding_dataset.filterDate('2024-01-01', '2025-01-01').mosaic()

# Split the SOC data into training (80%) and validation (20%) sets
soc_data = soc_data.randomColumn(columnName='random', seed=42)
training_fc = soc_data.filter(ee.Filter.lt('random', 0.8))
validation_fc = soc_data.filter(ee.Filter.gte('random', 0.8))

# Sample the embedding image at the locations of the SOC data
training = embedding_2024.sampleRegions(
    collection=training_fc, properties=['socd_100cm'], scale=10, tileScale=16
)
test = embedding_2024.sampleRegions(
    collection=validation_fc, properties=['socd_100cm'], scale=10, tileScale=16
)

# Tune the number of trees in Random Forest
band_names = embedding_2024.bandNames()
numTreesList = ee.List.sequence(0, 200, 20)

optimal_model = ee.Classifier.smileRandomForest(
    numberOfTrees=160,
    variablesPerSplit=8,
    minLeafPopulation=5,
    bagFraction=0.8,
    seed=42
).setOutputMode('REGRESSION').train(
    features=training,
    classProperty='socd_100cm',
    inputProperties=band_names
)

orgc_prediction = embedding_2024.classify(optimal_model)

In [10]:
prediction_vis = {
    'min': 0,
    'max': 50,
    'palette': [    
        '#543005', '#bc8120', '#dfc27d', '#f6e8c3',
        '#f5f5f5', '#c7eae5', '#80cdc1', '#35978f',
        '#01665e', '#003c30'
    ]
}
Map.addLayer(orgc_prediction, prediction_vis, 'SOCD Prediction')

In [11]:
Map

Map(bottom=100720.0, center=[38.749798086673444, 100.25000035704018], controls=(WidgetControl(options=['positi…

In [12]:
orgc_prediction_clipped = orgc_prediction.clip(selected_grid)

In [15]:
download_dir = r'D:\0_phd_papers\2_SOC_Aspect\9_plot\plot_r_distribution\plot_method\dem_ings'
geemap.download_ee_image(
    image=orgc_prediction_clipped,
    filename=os.path.join(download_dir, 'socd_grid_10882.tif'),
    scale=30,
    region=selected_grid.geometry(),
    crs='EPSG:4326',
    max_tile_size=16,
    max_tile_dim=512
)

socd_grid_10882.tif: |          | 0.00/13.8M (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None
